### Open in Jupyter Notebook (not in pycharm) to get notebook to display properly

In [21]:
%%HTML 
<!-- HTML Code to hide code cells -->
<script>
    function luc21893_refresh_cell(cell) {
        if( cell.luc21893 ) return;
        cell.luc21893 = true;
        console.debug('New code cell found...' );
        
        var div = document.createElement('DIV');            
        cell.parentNode.insertBefore( div, cell.nextSibling );
        div.style.textAlign = 'right';
        var a = document.createElement('A');
        div.appendChild(a);
        a.href='#'
        a.luc21893 = cell;
        a.setAttribute( 'onclick', "luc21893_toggle(this); return false;" );

        cell.style.visibility='hidden';
        cell.style.position='absolute';
        a.innerHTML = '[show code]';        
                
    }
    function luc21893_refresh() {                
        if( document.querySelector('.code_cell .input') == null ) {            
            // it apeears that I am in a exported html
            // hide this code
            var codeCells = document.querySelectorAll('.jp-InputArea')
            codeCells[0].style.visibility = 'hidden';
            codeCells[0].style.position = 'absolute';                        
            for( var i = 1; i < codeCells.length; i++ ) {
                luc21893_refresh_cell(codeCells[i].parentNode)
            }
            window.onload = luc21893_refresh;
        }                 
        else {
            // it apperas that I am in a jupyter editor
            var codeCells = document.querySelectorAll('.code_cell .input')
            for( var i = 0; i < codeCells.length; i++ ) {
                luc21893_refresh_cell(codeCells[i])
            }            
            window.setTimeout( luc21893_refresh, 1000 )
        }        
    }
    
    function luc21893_toggle(a) {
        if( a.luc21893.style.visibility=='hidden' ) {
            a.luc21893.style.visibility='visible';        
            a.luc21893.style.position='';
            a.innerHTML = '[hide code]';
        }
        else {
            a.luc21893.style.visibility='hidden';        
            a.luc21893.style.position='absolute';
            a.innerHTML = '[show code]';
        }
    }
    
    luc21893_refresh()
</script>

# General Biomass and Waste Gasification LCA

### TODO: Add more detailed Instructions here at later stage:


In [41]:
# Import required packages
import ipywidgets as widgets
from IPython.display import display, Javascript
from config import settings
from functions.general.utility.toml_handling import update_user_inputs_toml, reset_user_inputs_toml


In [42]:
"""
Improve this function - have button to click at end of each section after you're done with your inputs...

Alternative workflow for user - click run all button, then enter values - fields will be displayed automatically - then
click button saying start simulation - show progress bar
    - issue with this might currently be that some cells e.g. bed material are dependent on other inputs
    - could also have code for button at end of each cell where I want a button and just directly link to whatever cell we want.
        - i.e just say cell X in JavaScript - would not even need to fetch current cell number or anything. could just do manually
        
"""












# Create Buttons to progress to next Section when finished with previous section
from IPython.display import display

# Create layout object to style button
layout_button = widgets.Layout(width='auto', height='40px') #set width and height



button = widgets.Button(description="Start Simulation", # Click to proceed to next cell
                        layout = layout_button
                       ) 
output = widgets.Output() 
display(button, output) 

# Define function to execute next code cell - executed upon button click
# NOTE: Change to idx+2 if I decided to add a markdown cell between cells
def on_button_click(inp) : 
    with output:    
        # Define Javascript command to run next cell
        js_command = """
        var output_area = this;
        // find my cell element
        var cell_element = output_area.element.parents('.cell');
        // which cell is it?
        var cell_idx = Jupyter.notebook.get_cell_elements().index(cell_element);
        // get the cell object
        var cell = Jupyter.notebook.get_cell(cell_idx);

         // Execute next cell
        Jupyter.notebook.execute_cells([cell_idx+{}])
        """.format(2)
        
        # Execute command
        display(Javascript(js_command))
        
        # Tell user
        #print("Proceeded to next section")

button.on_click(on_button_click)


Button(description='Start Simulation', layout=Layout(height='40px', width='auto'), style=ButtonStyle())

Output()

### Please define general simulation parameters:

In [43]:
# Define MC widget
MC_widget = widgets.BoundedIntText(value = 1000,
                                   min=0,
                                   max=10000,
                                   step=100,
                                   description="Select the number of Monte Carlo iterations:",
                                   style={'description_width': 'initial'},
                                   layout={'width':'500px'}
                                  )

# Define country widget
country_widget = widgets.Dropdown(options = settings.general.countries,
                                  description="Select a reference country:",
                                  style={'description_width': 'initial'},
                                  layout={'width':'500px'}
                                 )


### Please provide information about the feedstock

In [44]:
# Feedstock category widget
feedstock_category_widget = widgets.Dropdown(options = settings.general.feedstock_categories,
                                             description="Select a feedstock category:",
                                             style={'description_width': 'initial'},
                                             layout={'width':'500px'}
                                            )

# Feedstock name widget
feedstock_name_widget = widgets.Textarea(placeholder='Enter feedstock name',
                                         description='Please enter a descriptive name of your feedstock:',
                                         style={'description_width': 'initial'},
                                         layout={'width':'500px'}
                                        )

# Feedstock particle size widget
particle_size_widget = widgets.BoundedFloatText(min=0,
                                                max=100,
                                                style={'description_width': 'initial'},
                                                layout={'width':'500px'},
                                                description = "Particle size [mm]:"
                                               )



# Feedstock elemental composition widgets
C_widget = widgets.BoundedFloatText(min=0,
                                    max=100,
                                    style={'description_width': 'initial'},
                                    layout={'width':'500px'},
                                    description = "Carbon content [%daf]:"
                                   )

H_widget = widgets.BoundedFloatText(min=0,
                                    max=100,
                                    style={'description_width': 'initial'},
                                    layout={'width':'500px'},
                                    description= "Hydrogen content [%daf]:"
                                   )

S_widget = widgets.BoundedFloatText(min=0,
                                    max=100,
                                    style={'description_width': 'initial'},
                                    layout={'width':'500px'},
                                    description=  "Sulphur content [%daf]:"
                                   )

# Feedstock moisture widgets
received_moisture_widget = widgets.BoundedFloatText(min=0,
                                                    max=100,
                                                    style={'description_width': 'initial'},
                                                    layout={'width':'500px'},
                                                    description=  "Feedstock moisture as received [%wb]:"
                                                   )

desired_moisture_widget = widgets.BoundedFloatText(min=0,
                                                   max=100,
                                                   style={'description_width': 'initial'},
                                                   layout={'width':'500px'},
                                                   description= "Desired feedstock moisture post drying [%wb]:"
                                                  )

ash_widget = widgets.BoundedFloatText(min=0,
                                      max=100,
                                      style={'description_width': 'initial'},
                                      layout={'width':'500px'},
                                      description= "Ash content [%db]:"
                                     )

# Display widgets
print("General feedstock information:")
display(feedstock_category_widget)
display(feedstock_name_widget)
display(particle_size_widget)

print("Feedstock elemental composition:")
display(C_widget)
display(H_widget)
display(S_widget)

print("Feedstock proximate composition:")
display(received_moisture_widget)
display(desired_moisture_widget)
display(ash_widget)


General feedstock information:


Dropdown(description='Select a feedstock category:', layout=Layout(width='500px'), options=('plastics', 'munic…

Textarea(value='', description='Please enter a descriptive name of your feedstock:', layout=Layout(width='500p…

BoundedFloatText(value=0.0, description='Particle size [mm]:', layout=Layout(width='500px'), style=Description…

Feedstock elemental composition:


...

BoundedFloatText(value=0.0, description='Sulphur content [%daf]:', layout=Layout(width='500px'), style=Descrip…

Feedstock proximate composition:


BoundedFloatText(value=0.0, description='Feedstock moisture as received [%wb]:', layout=Layout(width='500px'),…

BoundedFloatText(value=0.0, description='Desired feedstock moisture post drying [%wb]:', layout=Layout(width='…

BoundedFloatText(value=0.0, description='Ash content [%db]:', layout=Layout(width='500px'), style=DescriptionS…

In [45]:
if desired_moisture_widget.value > received_moisture_widget.value:
    raise ValueError("Desired feedstock moisture after drying should not be higher than as received moisture")

### Please provide information about reactor and operating conditions

In [47]:
# Operating condition widgets
temperature_widget = widgets.BoundedFloatText(min=0,
                                              max=1500,
                                              style={'description_width': 'initial'},
                                              layout={'width':'500px'},
                                              description = 'Gasification Temperature [°C]:'
                                             )
ER_widget = widgets.BoundedFloatText(value=0.3,
                                     min=0,
                                     max=1,
                                     style={'description_width': 'initial'},
                                     layout={'width':'500px'},
                                     description = 'Equivalence Ratio (ER):'
                                    )

operation_mode_widget = widgets.Dropdown(options = ['Continuous', 'Batch'],
                                         description="Select the operation mode:",
                                         style={'description_width': 'initial'},
                                         layout={'width':'500px'}
                                        )


operation_scale_widget = widgets.Dropdown(options = ['Lab', 'Pilot'],
                                          description="Select the gasifier scale:",
                                          style={'description_width': 'initial'},
                                          layout={'width':'500px'}
                                         )

gasifying_agent_widget = widgets.Dropdown(options = ["Air", "Air + steam", "Oxygen", "Steam", "Other"],
                                          description="Select a gasifying agent:",
                                          style={'description_width': 'initial'},
                                          layout={'width':'500px'}
                                         )

reactor_type_widget = widgets.Dropdown(options = ['Fixed bed', 'Fluidised bed', 'Other'],
                                       description="Select the reactor type:",
                                       style={'description_width': 'initial'},
                                       layout={'width':'500px'}
                                      )

catalyst_widget = widgets.Dropdown(options = ['No', 'Yes'],
                                   description="Is a catalyst used?",
                                   style={'description_width': 'initial'},
                                   layout={'width':'500px'}
                                  )

# Display widgets
display(temperature_widget)
display(ER_widget)
display(operation_mode_widget)
display(catalyst_widget)
display(operation_scale_widget)
display(gasifying_agent_widget)
display(reactor_type_widget)


BoundedFloatText(value=0.0, description='Gasification Temperature [°C]:', layout=Layout(width='500px'), max=15…

BoundedFloatText(value=0.3, description='Equivalence Ratio (ER):', layout=Layout(width='500px'), max=1.0, styl…

Dropdown(description='Select the operation mode:', layout=Layout(width='500px'), options=('Continuous', 'Batch…

Dropdown(description='Is a catalyst used?', layout=Layout(width='500px'), options=('No', 'Yes'), style=Descrip…

Dropdown(description='Select the gasifier scale:', layout=Layout(width='500px'), options=('Lab', 'Pilot'), sty…

Dropdown(description='Select a gasifying agent:', layout=Layout(width='500px'), options=('Air', 'Air + steam',…

Dropdown(description='Select the reactor type:', layout=Layout(width='500px'), options=('Fixed bed', 'Fluidise…

In [48]:
bed_material_widget = widgets.Dropdown(options = ["Silica", "Alumina", "Olivine", "Other"],
                                             description="Select a bed material:",
                                             style={'description_width': 'initial'},
                                             layout={'width':'500px'}
                                            )

# Display bed material widget if reactor type other than fixed bed was selected
if reactor_type_widget.value != 'Fixed bed':
    display(bed_material_widget)

Dropdown(description='Select a bed material:', layout=Layout(width='500px'), options=('Silica', 'Alumina', 'Ol…

In [49]:
# Reset .toml file with user inputs and prepare for new entries
reset_user_inputs_toml()


# Write general simulation parameters to .toml file
update_user_inputs_toml('iterations_MC', MC_widget.value)
update_user_inputs_toml('country', country_widget.value)


# Write feedstock information to .toml file
# General feedstock information
update_user_inputs_toml('feedstock_category', feedstock_category_widget.value)
update_user_inputs_toml('feedstock_name', feedstock_name_widget.value)
update_user_inputs_toml("particle size",particle_size_widget.value)

# Feedstock elemental composition
update_user_inputs_toml('carbon content', C_widget.value)
update_user_inputs_toml('hydrogen content', H_widget.value)
update_user_inputs_toml('sulphur content', S_widget.value)

# Feedstock proximate composition
update_user_inputs_toml('as received moisture', received_moisture_widget.value)
update_user_inputs_toml('desired moisture', desired_moisture_widget.value)
update_user_inputs_toml('ash', ash_widget.value)


# Write operating conditions to .toml file
update_user_inputs_toml('temperature', temperature_widget.value)
update_user_inputs_toml('ER', ER_widget.value)
update_user_inputs_toml('operation mode', operation_mode_widget.value)
update_user_inputs_toml('operation scale', operation_scale_widget.value)
update_user_inputs_toml('gasifying agent', gasifying_agent_widget.value)

# Convert catalyst use before storing in toml
if catalyst_widget.value == 'Yes':
    catalyst_value_to_toml = True
else:
    catalyst_value_to_toml = False
update_user_inputs_toml('catalyst', catalyst_value_to_toml)

# Convert reactor type before storing in toml
if reactor_type_widget.value == 'Fixed bed':
    reactor_type_value_to_toml = 'Fixed'
elif reactor_type_widget.value == 'Fluidised bed':
    reactor_type_value_to_toml = 'Fluidised'
else:
    reactor_type_value_to_toml = 'Other'
update_user_inputs_toml('reactor type', reactor_type_value_to_toml)

# Store bed material based on reactor type
if reactor_type_widget.value == 'Fixed bed':
    update_user_inputs_toml('bed material', 'N/A')
else:
    update_user_inputs_toml('bed material', bed_material_widget.value)


### Thanks your LCA results will be calculated.


In [50]:
# Execute LCA
# figure out how to show loading progress bar - Jan has it in his jupyter notebook - can ask him about it.


